# Bijgebouw Comparison

In [31]:
import pandas as pd
import numpy as np
import folium
from branca.element import Figure
from folium.plugins import MarkerCluster

In [34]:
df1 = pd.read_excel(r"C:\Users\Gebruiker\Desktop\GITHUB\Geomatics\book21_edited.xlsx")

df1.head()

,Nummer,H,Address,Latitude,Longitude
0,210004,het plaatsen van een dakkapel,"1, Zeistpad, Stedenwijk, Almere Stad, Almere, ...",52.363841,5.203593
1,210039,het plaatsen van een dakkapel,"34, Barneveldpad, Stedenwijk, Almere Stad, Alm...",52.362509,5.208172
2,210092,het plaatsen van een dakkapel,"35, Breitnerstraat, Tussen de Vaarten, Almere,...",52.379402,5.254923
3,210172,het plaatsen van een dakkapel,"12, Beuvenstraat, Waterwijk, Almere, Flevoland...",52.386438,5.221712
4,210227,Het plaatsen van 2 dakkapellen,"8, Kaatsheuvelstraat, Stedenwijk, Almere, Flev...",52.357661,5.201078


In [35]:
locations = df1[['Latitude','Longitude']]
locationlist = locations.values.tolist()
len(locationlist)
locationlist[7]

[52.3885517, 5.2397798]

In [50]:
loc = 'Bijgebouw Aanvragen 2021'
title_html = '''
             <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(loc)   

map = folium.Map(location=[52.371320, 5.222381], zoom_start=12)
for point in range(0, len(locationlist)):
    folium.Marker(locationlist[point], popup=df1['H'][point]).add_to(map)


In [51]:
def temp(map):
    fig2=Figure(width=1000,height=800)
    fig2.add_child(map)
    folium.TileLayer('Stamen Terrain').add_to(map)
    folium.TileLayer('Stamen Toner').add_to(map)
    folium.TileLayer('Stamen Water Color').add_to(map)
    folium.TileLayer('cartodbpositron').add_to(map)
    folium.TileLayer('cartodbdark_matter').add_to(map)
    folium.LayerControl().add_to(map)
    map.get_root().html.add_child(folium.Element(title_html))




In [53]:
# map.save('map-with-title.html')
temp(map)
map

In [28]:

map2 = folium.Map(location=[52.371320, 5.222381], tiles='CartoDB dark_matter', zoom_start=11)


marker_cluster = MarkerCluster().add_to(map2)

for point in range(0, len(locationlist)):
    folium.Marker(locationlist[point], popup=df1['H'][point]).add_to(marker_cluster)


temp(map2)

In [30]:
map2 = folium.Map(location=[52.371320, 5.222381], tiles='Stamen Terrain', zoom_start=11)

marker_cluster = MarkerCluster().add_to(map2)

for point in range(0, len(locationlist)):
    folium.Marker(locationlist[point], popup=df1['H'][point], icon=folium.Icon(color='darkblue', icon_color='white', icon='male', angle=0, prefix='fa')).add_to(marker_cluster)

map2

The terrain map is an improvement over the dark map, but I still found the green coloring for natural areas to be distracting from the main objective of the map. I now also want to differentiate the counters by region. To do that, I decided to create a function to assign a unique color to each region.

In [21]:
def regioncolors(df1):
    if df1['H'] == 'het plaatsen van een dakkapel':
        return 'green'
    elif df1['H'] == 'Het plaatsen van aanbouw':
        return 'blue'
    elif df1['H'] == 'het plaatsen van een dakopbouw':
        return 'red'
    else:
        return 'darkblue'
df1["color"] = df1.apply(regioncolors, axis=1)
df1.head()

,Nummer,H,Address,Latitude,Longitude,color
0,210004,het plaatsen van een dakkapel,"1, Zeistpad, Stedenwijk, Almere Stad, Almere, ...",52.363841,5.203593,darkblue
1,210039,het plaatsen van een dakkapel,"34, Barneveldpad, Stedenwijk, Almere Stad, Alm...",52.362509,5.208172,green
2,210092,het plaatsen van een dakkapel,"35, Breitnerstraat, Tussen de Vaarten, Almere,...",52.379402,5.254923,green
3,210172,het plaatsen van een dakkapel,"12, Beuvenstraat, Waterwijk, Almere, Flevoland...",52.386438,5.221712,green
4,210227,Het plaatsen van 2 dakkapellen,"8, Kaatsheuvelstraat, Stedenwijk, Almere, Flev...",52.357661,5.201078,darkblue


I then changed the point icon color for each counter based on its region. I also switched the basemap to the lighter CartoDB positron tiles. While I was at it, I also realized it would be a really good idea to include the counter ID number on the map since you need that number when making specific API calls for counter data.

In [27]:
map3 = folium.Map(location=[52.371320, 5.222381], tiles='CartoDB positron', zoom_start=11)

marker_cluster = MarkerCluster().add_to(map3)

for point in range(0, len(locationlist)):
    folium.Marker(locationlist[point], popup='ID:'+str(df1['Nummer'][point])+' '+df1['H'][point], icon=folium.Icon(color=df1["color"][point], icon_color='white', icon='male', angle=0, prefix='fa')).add_to(marker_cluster)
map3

I'm fairly please with this last map. In the future, I would like to change the symbols on the pointers based on whether the particular sensor can distiguish between bicycles and pedestrians. It is also possible to attach bar charts or other graph types to these pointers. In addition to point maps, Folium can also create choropleth maps for displaying regional statistics using color gradients.